In [520]:
# 本代码由可视化策略环境自动生成 2021年4月26日18:02
# 本代码单元只能在可视化模式下编辑。您也可以拷贝代码，粘贴到新建的代码单元或者策略，然后修改。


# Python 代码入口函数，input_1/2/3 对应三个输入端，data_1/2/3 对应三个输出端
def m11_run_bigquant_run(input_1, input_2, input_3):
    # 示例代码如下。在这里编写您的代码
    dt = input_1.read_df().sort_values(by='date',ascending=False).reset_index(drop=True)
    dt1 = dt.groupby('instrument')
    stock_list = dt['instrument'].value_counts().index
    dt2 = pd.DataFrame()
    for stock in stock_list:
        dd = dt1.get_group(stock)
        dd1 = ((dd['close_0'].rolling(44).mean() / dd['close_0'])-1)
        dt2 = pd.concat([dt2,dd1],ignore_index=True)
    dt2.columns=['factor']
    dt3 = pd.concat([dt,dt2],axis=1)
    dt3 = dt3[['factor','date','instrument']]
    data_1 = DataSource.write_df(dt3)
    return Outputs(data_1=data_1, data_2=None, data_3=None)
# 后处理函数，可选。输入是主函数的输出，可以在这里对数据做处理，或者返回更友好的outputs数据格式。此函数输出不会被缓存。
def m11_post_run_bigquant_run(outputs):
    return outputs

# 回测引擎：初始化函数，只执行一次
def m10_initialize_bigquant_run(context):
    context.indicator_data = context.options['data'].read_df()
    context.set_commission(PerOrder(buy_cost=0.0001, sell_cost=0.001, min_cost=5))
    context.stock_num = 50
    context.rebalance_days = 22
    if 'index' not in context.extension:
        context.extension['index'] = 0
 
    

# 回测引擎：每日数据处理函数，每天执行一次
def m10_handle_data_bigquant_run(context, data):
    #------------------------------------------止损模块START--------------------------------------------
    date = data.current_dt.strftime('%Y-%m-%d')
    positions = {e.symbol: p.cost_basis  for e, p in context.portfolio.positions.items()}
    current_stoploss_stock = [] 
    if len(positions) > 0:
        for i in positions.keys():
            stock_cost = positions[i] 
            stock_market_price = data.current(context.symbol(i), 'price') 
            if (stock_market_price - stock_cost) / stock_cost <= -0.1:   
                context.order_target_percent(context.symbol(i),0)     
                current_stoploss_stock.append(i)
                print('日期：',date,'股票：',i,'出现止损状况')
    #-------------------------------------------止损模块END--------------------------------------------------    
    context.extension['index'] += 1
    if  context.extension['index'] % context.rebalance_days != 0:
        return 
    
    # 当前的日期
    date = data.current_dt.strftime('%Y-%m-%d')
    
    cur_data = context.indicator_data[context.indicator_data['date'] == date]
    # 根据日期获取调仓需要买入的股票的列表
    stock_to_buy = list(cur_data.instrument[:context.stock_num])
    # 通过positions对象，使用列表生成式的方法获取目前持仓的股票列表
    stock_hold_now = [equity.symbol for equity in context.portfolio.positions]
    # 继续持有的股票：调仓时，如果买入的股票已经存在于目前的持仓里，那么应继续持有
    no_need_to_sell = [i for i in stock_hold_now if i in stock_to_buy]
    # 需要卖出的股票
    stock_to_sell = [i for i in stock_hold_now if i not in no_need_to_sell]
  
    # 卖出
    for stock in stock_to_sell:
        if stock in current_stoploss_stock:
            continue
        if data.can_trade(context.symbol(stock)):
            context.order_target_percent(context.symbol(stock), 0)
            
    # 如果当天没有买入的股票，就返回
    if len(stock_to_buy) == 0:
        return
    
    weight =  1 / len(stock_to_buy)    
    # 买入
    for stock in stock_to_buy:
        if data.can_trade(context.symbol(stock)):
            context.order_target_percent(context.symbol(stock), weight)
 
# 回测引擎：准备数据，只执行一次
def m10_prepare_bigquant_run(context):
    pass

# 回测引擎：每个单位时间开始前调用一次，即每日开盘前调用一次。
def m10_before_trading_start_bigquant_run(context, data):
    pass


m1 = M.instruments.v2(
    start_date='2020-01-01',
    end_date='2021-01-01',
    market='CN_STOCK_A',
    instrument_list='',
    max_count=0
)

m3 = M.input_features.v1(
    features="""
# #号开始的表示注释，注释需单独一行
# 多个特征，每行一个，可以包含基础特征和衍生特征，特征须为本平台特征
close_0"""
)

m7 = M.general_feature_extractor.v7(
    instruments=m1.data,
    features=m3.data,
    start_date='',
    end_date='',
    before_start_days=90
)

m11 = M.cached.v3(
    input_1=m7.data,
    run=m11_run_bigquant_run,
    post_run=m11_post_run_bigquant_run,
    input_ports='',
    params='{}',
    output_ports=''
)

m9 = M.dropnan.v2(
    input_data=m11.data_1
)

m8 = M.sort.v4(
    input_ds=m9.data,
    sort_by='factor',
    group_by='date',
    keep_columns='--',
    ascending=True
)

m10 = M.trade.v4(
    instruments=m1.data,
    options_data=m8.sorted_data,
    start_date='',
    end_date='',
    initialize=m10_initialize_bigquant_run,
    handle_data=m10_handle_data_bigquant_run,
    prepare=m10_prepare_bigquant_run,
    before_trading_start=m10_before_trading_start_bigquant_run,
    volume_limit=0.025,
    order_price_field_buy='open',
    order_price_field_sell='open',
    capital_base=1000000,
    auto_cancel_non_tradable_orders=True,
    data_frequency='daily',
    price_type='后复权',
    product_type='股票',
    plot_charts=True,
    backtest_only=False,
    benchmark=''
)

m6 = M.input_features.v1(
    features="""
# #号开始的表示注释，注释需单独一行
# 多个特征，每行一个，可以包含基础特征和衍生特征，特征须为本平台特征
factor
"""
)

m4 = M.factorlens.v1(
    features=m6.data,
    user_factor_data=m11.data_1,
    title='因子分析: {factor_name}',
    start_date='2020-01-01',
    end_date='2021-01-01',
    rebalance_period=22,
    delay_rebalance_days=0,
    rebalance_price='close_0',
    stock_pool='全市场',
    quantile_count=5,
    commission_rate=0.0016,
    returns_calculation_method='累乘',
    benchmark='无',
    drop_price_limit_stocks=True,
    drop_st_stocks=True,
    drop_new_stocks=True,
    normalization=True,
    neutralization=[],
    metrics=['因子表现概览', '因子分布', '因子行业分布', '因子市值分布', 'IC分析', '买入信号重合分析', '因子估值分析', '因子拥挤度分析', '因子值最大/最小股票', '表达式因子值', '多因子相关性分析'],
    factor_coverage=0.5,
    user_data_merge='left'
)


[2021-04-26 17:58:55.213496] INFO: moduleinvoker: instruments.v2 开始运行..

[2021-04-26 17:58:55.225612] INFO: moduleinvoker: 命中缓存

[2021-04-26 17:58:55.228075] INFO: moduleinvoker: instruments.v2 运行完成[0.01458s].

[2021-04-26 17:58:55.231868] INFO: moduleinvoker: input_features.v1 开始运行..

[2021-04-26 17:58:55.245628] INFO: moduleinvoker: 命中缓存

...

[2021-04-26 17:59:44.746647] INFO: moduleinvoker: 命中缓存

[2021-04-26 17:59:44.749626] INFO: moduleinvoker: input_features.v1 运行完成[0.016145s].

[2021-04-26 17:59:44.756394] INFO: moduleinvoker: factorlens.v1 开始运行..

[2021-04-26 17:59:44.764559] INFO: moduleinvoker: 命中缓存

[2021-04-26 17:59:44.942515] INFO: moduleinvoker: factorlens.v1 运行完成[0.186107s].